## TABLA PREDICTORA
Este notebook tiene como objetivo recoger los 4 csv (en los que he ido agrupando los datos según su origen) y crear las funciones neceesarias para generar el df predictor con el que comenzaré a entrenar el modelo.

A modo de resumen, los csv en cuestión y sus procedencias y contenidos son:
* **ataque_ratings**: viene de la página de 2kratings, y contienen información (por temporada) del potencial de los equipo en ataque.
* **overall_ratings**:  viene de la página de 2kratings, y contienen información (por temporada) del potencial de los equipo, y una clasificación de los equipos en TIERs.
* **indicadores**:  viene de la página de ESPN. Contiene también información por temporada sobre el potencial de los equipos. Esta información es redundante al csv overall_ratings, pero como viene de fuentes diferentes voy a dejarlos para posteriormente estudiar la correlación ue tienen con el target y deshacerme en el feature selection del las columnas que menos capacidad predictiva me den.
* **partidos_24_limpio**: viene de la API nba_api (tb en librería de python), contiene información de todos los partidos jugados en la temporada actual. Lo usaré para generar las features que indicarán el desempeño de los equipos en las útlimas jornadas. 



### Importar librerías necesarias

In [33]:
import pandas as pd

### Carga de csv

In [59]:
ataque_df = pd.read_csv(r'C:\Users\iauzm\OneDrive\Desktop\DATA SCIENCE\AUZMENDI\PROYECTO-ML\data_limpia\ataque_ratings.csv',index_col='Unnamed: 0')
indicadores_df = pd.read_csv(r'C:\Users\iauzm\OneDrive\Desktop\DATA SCIENCE\AUZMENDI\PROYECTO-ML\data_limpia\indicadores.csv',index_col='Unnamed: 0')
general_df = pd.read_csv(r'C:\Users\iauzm\OneDrive\Desktop\DATA SCIENCE\AUZMENDI\PROYECTO-ML\data_limpia\overall_ratings.csv',index_col='Unnamed: 0')
partidos_df = pd.read_csv(r'C:\Users\iauzm\OneDrive\Desktop\DATA SCIENCE\AUZMENDI\PROYECTO-ML\data_limpia\partidos_24_limpio',index_col='Unnamed: 0')

In [60]:
# Imprime el head de ataque_df
print("Ataque:")
display(ataque_df.head())

# Imprime el head de indicadores_df
print("Indicadores:")
display(indicadores_df.head())

# Imprime el head de general_df
print("General:")
display(general_df.head())

# Imprime el head de partidos_df
print("partidos:")
display(partidos_df.head())

Ataque:


,Team,INS,OUT
0,Atlanta Hawks,61,82
1,Boston Celtics,81,91
2,Brooklyn Nets,68,77
3,Charlotte Hornets,66,81
4,Chicago Bulls,76,85


Indicadores:


,team_name,abbreviation,bpi,off,def,pbpi,wins,loses
0,Celtics,BOS,8.075,6.593,1.482,8.075,75,20
1,Nuggets,DEN,4.684,2.487,2.197,4.684,64,30
2,Thunder,OKC,5.060,2.404,2.657,5.060,63,29
3,Timberwolves,MIN,4.728,1.192,3.536,4.728,64,32
4,Clippers,LAC,3.726,3.481,0.245,3.726,53,35


General:


,Team,OVR,DEF,TIER_T1,TIER_T2,TIER_T3
0,Atlanta Hawks,82,64,False,False,True
1,Boston Celtics,86,69,True,False,False
2,Brooklyn Nets,80,66,False,False,True
3,Charlotte Hornets,80,61,False,False,True
4,Chicago Bulls,82,66,False,False,True


partidos:


,abreviación_eq_local,equipo_local,fecha,cabecera,WL_local,pts_local,FGM_local,FGA_local,FG_PCT_local,FG3M_local,...,FT_PCT_visitante,OREB_visitante,DREB_visitante,REB_visitante,AST_visitante,STL_visitante,BLK_visitante,TOV_visitante,PF_visitante,PLUS_MINUS_visitante
0,DEN,Denver Nuggets,2023-10-24,DEN VS LAL,W,119.0,48.0,91.0,0.527,14.0,...,0.750,13.0,31.0,44.0,23.0,5.0,4.0,11.0,18.0,-12.0
1,GSW,Golden State Warriors,2023-10-24,GSW VS PHX,L,104.0,36.0,101.0,0.356,10.0,...,0.765,17.0,43.0,60.0,23.0,5.0,7.0,19.0,22.0,4.0
2,BKN,Brooklyn Nets,2023-10-25,BKN VS CLE,L,113.0,42.0,84.0,0.500,9.0,...,0.900,9.0,29.0,38.0,25.0,10.0,6.0,10.0,23.0,1.0
3,CHA,Charlotte Hornets,2023-10-25,CHA VS ATL,W,116.0,43.0,86.0,0.500,11.0,...,0.818,12.0,30.0,42.0,24.0,12.0,1.0,12.0,19.0,-6.0
4,CHI,Chicago Bulls,2023-10-25,CHI VS OKC,L,104.0,39.0,94.0,0.415,12.0,...,0.833,5.0,38.0,43.0,30.0,6.0,4.0,15.0,19.0,20.0


### Merge de los df, con información general de los equipos. 
Voy a dividir el proceso de merge en 2 partes. 1º voy a hacer el merge de los 3 primeros dfs. Estos dfs deberían tener una longitud de 30 ( 1 row por equipo). Y debería poder hacer el mege sobre la columna con el nombre del equipo sin más problema. 

Antes de eso, el df General tiene como nombre del equipo la palabra que no hace referencia a la ciudad del equipo. Voy a hacer un split en los otros dos primero para poder hacer el merge.

In [61]:
indicadores_df.shape


(30, 8)

In [62]:
(ataque_df['Team'])

0              Atlanta Hawks
1             Boston Celtics
2              Brooklyn Nets
3          Charlotte Hornets
4              Chicago Bulls
5        Cleveland Cavaliers
6           Dallas Mavericks
7             Denver Nuggets
8            Detroit Pistons
9      Golden State Warriors
10           Houston Rockets
11            Indiana Pacers
12      Los Angeles Clippers
13        Los Angeles Lakers
14         Memphis Grizzlies
15                Miami Heat
16           Milwaukee Bucks
17    Minnesota Timberwolves
18      New Orleans Pelicans
19           New York Knicks
20     Oklahoma City Thunder
21             Orlando Magic
22        Philadelphia 76ers
23              Phoenix Suns
24    Portland Trail Blazers
25          Sacramento Kings
26         San Antonio Spurs
27           Toronto Raptors
28                 Utah Jazz
29        Washington Wizards
Name: Team, dtype: object

In [63]:
names_map= {'Atlanta Hawks':'Hawks',
 'Boston Celtics':'Celtics',
 'Brooklyn Nets':'Nets',
 'Charlotte Hornets':'Hornets',
 'Chicago Bulls':'Bulls',
 'Cleveland Cavaliers':'Cavaliers',
 'Dallas Mavericks':'Mavericks',
 'Denver Nuggets':'Nuggets',
 'Detroit Pistons':'Pistons',
 'Golden State Warriors':'Warriors',
 'Houston Rockets':'Rockets',
 'Indiana Pacers':'Pacers',
 'Los Angeles Clippers':'Clippers',
 'Los Angeles Lakers':'Lakers',
 'Memphis Grizzlies':'Grizzlies',
 'Miami Heat':'Heat',
 'Milwaukee Bucks':'Bucks',
 'Minnesota Timberwolves':'Timberwolves',
 'New Orleans Pelicans':'Pelicans',
 'New York Knicks':'Knicks',
 'Oklahoma City Thunder':'Thunder',
 'Orlando Magic':'Magic',
 'Philadelphia 76ers':'76ers',
 'Phoenix Suns':'Suns',
 'Portland Trail Blazers':'Trail Blazers',
 'Sacramento Kings':'Kings',
 'San Antonio Spurs':'Spurs',
 'Toronto Raptors':'Raptors',
 'Utah Jazz':'Jazz',
 'Washington Wizards':'Wizards'}
ataque_df['Team_full']=ataque_df['Team']
ataque_df['Team'] = ataque_df['Team'].map(names_map)
ataque_df.rename(columns={'Team':'team_name'})
general_df['Team_full']=general_df['Team']
general_df['Team'] = general_df['Team'].map(names_map)
general_df.rename(columns={'Team':'team_name'})

,team_name,OVR,DEF,TIER_T1,TIER_T2,TIER_T3,Team_full
0,Hawks,82,64,False,False,True,Atlanta Hawks
1,Celtics,86,69,True,False,False,Boston Celtics
2,Nets,80,66,False,False,True,Brooklyn Nets
3,Hornets,80,61,False,False,True,Charlotte Hornets
4,Bulls,82,66,False,False,True,Chicago Bulls
5,Cavaliers,82,66,True,False,False,Cleveland Cavaliers
6,Mavericks,83,64,False,True,False,Dallas Mavericks
7,Nuggets,83,65,False,True,False,Denver Nuggets
8,Pistons,79,63,False,False,True,Detroit Pistons
9,Warriors,82,66,False,True,False,Golden State Warriors


In [68]:
general_df.rename(columns={'Team':'team_name'},inplace=True)
general_df.head()

,team_name,OVR,DEF,TIER_T1,TIER_T2,TIER_T3,Team_full
0,Hawks,82,64,False,False,True,Atlanta Hawks
1,Celtics,86,69,True,False,False,Boston Celtics
2,Nets,80,66,False,False,True,Brooklyn Nets
3,Hornets,80,61,False,False,True,Charlotte Hornets
4,Bulls,82,66,False,False,True,Chicago Bulls


In [71]:
ataque_df.rename(columns={'Team':'team_name'},inplace=True)
ataque_df.head()

,team_name,INS,OUT,Team_full
0,Hawks,61,82,Atlanta Hawks
1,Celtics,81,91,Boston Celtics
2,Nets,68,77,Brooklyn Nets
3,Hornets,66,81,Charlotte Hornets
4,Bulls,76,85,Chicago Bulls


In [73]:
#HAGO EL MERGE
merge_df_1 = ataque_df.merge(indicadores_df,on=['team_name'])
merge_df_2 = merge_df_1.merge(general_df,on=['team_name'])
print(merge_df_1.shape)
print(merge_df_2.shape)

(30, 11)
(30, 17)


Me voy  deshacer de las columnas que extra que se han generado y no necesito y guardo en un csv nuevo.

In [79]:
merge_df = merge_df_2
merge_df.rename(columns={'Team_full_x':'Team_full'},inplace=True)
merge_df.drop(columns=['Team_full_y','team_name'],inplace=True)
merge_df

,team_name,INS,OUT,Team_full,abbreviation,bpi,off,def,pbpi,wins,loses,OVR,DEF,TIER_T1,TIER_T2,TIER_T3
0,Hawks,61,82,Atlanta Hawks,ATL,-1.485,0.922,-2.407,-1.485,36,46,82,64,False,False,True
1,Celtics,81,91,Boston Celtics,BOS,8.075,6.593,1.482,8.075,75,20,86,69,True,False,False
2,Nets,68,77,Brooklyn Nets,BKN,-4.156,-3.599,-0.558,-4.156,32,50,80,66,False,False,True
3,Hornets,66,81,Charlotte Hornets,CHA,-7.752,-4.775,-2.977,-7.752,21,61,80,61,False,False,True
4,Bulls,76,85,Chicago Bulls,CHI,-0.836,0.598,-1.434,-0.836,39,43,82,66,False,False,True
5,Cavaliers,74,82,Cleveland Cavaliers,CLE,3.055,0.600,2.456,3.055,53,41,82,66,True,False,False
6,Mavericks,75,87,Dallas Mavericks,DAL,6.116,3.479,2.637,6.116,61,36,83,64,False,True,False
7,Nuggets,78,85,Denver Nuggets,DEN,4.684,2.487,2.197,4.684,64,30,83,65,False,True,False
8,Pistons,74,76,Detroit Pistons,DET,-10.147,-6.258,-3.890,-10.147,14,68,79,63,False,False,True
9,Warriors,60,87,Golden State Warriors,GS,1.327,1.580,-0.253,1.327,46,36,82,66,False,True,False


In [81]:
merge_df

,INS,OUT,Team_full,abbreviation,bpi,off,def,pbpi,wins,loses,OVR,DEF,TIER_T1,TIER_T2,TIER_T3
0,61,82,Atlanta Hawks,ATL,-1.485,0.922,-2.407,-1.485,36,46,82,64,False,False,True
1,81,91,Boston Celtics,BOS,8.075,6.593,1.482,8.075,75,20,86,69,True,False,False
2,68,77,Brooklyn Nets,BKN,-4.156,-3.599,-0.558,-4.156,32,50,80,66,False,False,True
3,66,81,Charlotte Hornets,CHA,-7.752,-4.775,-2.977,-7.752,21,61,80,61,False,False,True
4,76,85,Chicago Bulls,CHI,-0.836,0.598,-1.434,-0.836,39,43,82,66,False,False,True
5,74,82,Cleveland Cavaliers,CLE,3.055,0.600,2.456,3.055,53,41,82,66,True,False,False
6,75,87,Dallas Mavericks,DAL,6.116,3.479,2.637,6.116,61,36,83,64,False,True,False
7,78,85,Denver Nuggets,DEN,4.684,2.487,2.197,4.684,64,30,83,65,False,True,False
8,74,76,Detroit Pistons,DET,-10.147,-6.258,-3.890,-10.147,14,68,79,63,False,False,True
9,60,87,Golden State Warriors,GS,1.327,1.580,-0.253,1.327,46,36,82,66,False,True,False


In [82]:
merge_df.to_csv('datos_generales')

### Creación del df con datos dependientes de  los últimos partidos
#### A TENER EN CUENTA:
* Haré dos dfs, uno con información de los últimos 3 partidos y otro con los últimos 5 para cada equipo. Para probar posteriormente cual me da mayor correlación con el target.


In [83]:
partidos_df.head()

,abreviación_eq_local,equipo_local,fecha,cabecera,WL_local,pts_local,FGM_local,FGA_local,FG_PCT_local,FG3M_local,...,FT_PCT_visitante,OREB_visitante,DREB_visitante,REB_visitante,AST_visitante,STL_visitante,BLK_visitante,TOV_visitante,PF_visitante,PLUS_MINUS_visitante
0,DEN,Denver Nuggets,2023-10-24,DEN VS LAL,W,119.0,48.0,91.0,0.527,14.0,...,0.750,13.0,31.0,44.0,23.0,5.0,4.0,11.0,18.0,-12.0
1,GSW,Golden State Warriors,2023-10-24,GSW VS PHX,L,104.0,36.0,101.0,0.356,10.0,...,0.765,17.0,43.0,60.0,23.0,5.0,7.0,19.0,22.0,4.0
2,BKN,Brooklyn Nets,2023-10-25,BKN VS CLE,L,113.0,42.0,84.0,0.500,9.0,...,0.900,9.0,29.0,38.0,25.0,10.0,6.0,10.0,23.0,1.0
3,CHA,Charlotte Hornets,2023-10-25,CHA VS ATL,W,116.0,43.0,86.0,0.500,11.0,...,0.818,12.0,30.0,42.0,24.0,12.0,1.0,12.0,19.0,-6.0
4,CHI,Chicago Bulls,2023-10-25,CHI VS OKC,L,104.0,39.0,94.0,0.415,12.0,...,0.833,5.0,38.0,43.0,30.0,6.0,4.0,15.0,19.0,20.0


In [85]:
partidos_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1266 entries, 0 to 1265
Data columns (total 46 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   abreviación_eq_local         1266 non-null   object 
 1   equipo_local                 1266 non-null   object 
 2   fecha                        1266 non-null   object 
 3   cabecera                     1266 non-null   object 
 4   WL_local                     1266 non-null   object 
 5   pts_local                    1266 non-null   float64
 6   FGM_local                    1266 non-null   float64
 7   FGA_local                    1266 non-null   float64
 8   FG_PCT_local                 1266 non-null   float64
 9   FG3M_local                   1266 non-null   float64
 10  FG3A_local                   1266 non-null   float64
 11  FG3_PCT_local                1266 non-null   float64
 12  FTM_local                    1266 non-null   float64
 13  FTA_local              

In [86]:
predictor_df =partidos_df[['fecha','equipo_local','equipo_visitante']]
predictor_df.head()

,fecha,equipo_local,equipo_visitante
0,2023-10-24,Denver Nuggets,Los Angeles Lakers
1,2023-10-24,Golden State Warriors,Phoenix Suns
2,2023-10-25,Brooklyn Nets,Cleveland Cavaliers
3,2023-10-25,Charlotte Hornets,Atlanta Hawks
4,2023-10-25,Chicago Bulls,Oklahoma City Thunder


In [93]:
partidos_df.columns

Index(['abreviación_eq_local', 'equipo_local', 'fecha', 'cabecera', 'WL_local',
       'pts_local', 'FGM_local', 'FGA_local', 'FG_PCT_local', 'FG3M_local',
       'FG3A_local', 'FG3_PCT_local', 'FTM_local', 'FTA_local', 'FT_PCT_local',
       'OREB_local', 'DREB_local', 'REB_local', 'AST_local', 'STL_local',
       'BLK_local', 'TOV_local', 'PF_local', 'PLUS_MINUS_local',
       'TEAM_ABBREVIATION_visitante', 'equipo_visitante', 'WL_visitante',
       'pts_visitante', 'FGM_visitante', 'FGA_visitante', 'FG_PCT_visitante',
       'FG3M_visitante', 'FG3A_visitante', 'FG3_PCT_visitante',
       'FTM_visitante', 'FTA_visitante', 'FT_PCT_visitante', 'OREB_visitante',
       'DREB_visitante', 'REB_visitante', 'AST_visitante', 'STL_visitante',
       'BLK_visitante', 'TOV_visitante', 'PF_visitante',
       'PLUS_MINUS_visitante'],
      dtype='object')

In [92]:
predictor_df['fecha']= pd.to_datetime(predictor_df['fecha'])
partidos_df['fecha']= pd.to_datetime(partidos_df['fecha'])

In [117]:
import datetime
fecha= predictor_df['fecha'].iloc[2]
equipo_local= 'Brooklyn Nets'

partidos_brooklyn=partidos_df[(partidos_df['fecha']<fecha) & ((partidos_df['equipo_local']==equipo_local) | (partidos_df['equipo_visitante']==equipo_local))]

In [118]:
partidos_brooklyn

,abreviación_eq_local,equipo_local,fecha,cabecera,WL_local,pts_local,FGM_local,FGA_local,FG_PCT_local,FG3M_local,...,FT_PCT_visitante,OREB_visitante,DREB_visitante,REB_visitante,AST_visitante,STL_visitante,BLK_visitante,TOV_visitante,PF_visitante,PLUS_MINUS_visitante


In [99]:
partidos_brooklyn[partidos_brooklyn['fecha']<fecha]

,abreviación_eq_local,equipo_local,fecha,cabecera,WL_local,pts_local,FGM_local,FGA_local,FG_PCT_local,FG3M_local,...,FT_PCT_visitante,OREB_visitante,DREB_visitante,REB_visitante,AST_visitante,STL_visitante,BLK_visitante,TOV_visitante,PF_visitante,PLUS_MINUS_visitante


In [120]:
         WL_local ='WL_local'
       'pts_local', 'FGM_local', 'FGA_local', 'FG_PCT_local', 'FG3M_local',
       'FG3A_local', 'FG3_PCT_local', 'FTM_local', 'FTA_local', 'FT_PCT_local',
       'OREB_local', 'DREB_local', 'REB_local', 'AST_local', 'STL_local',
       'BLK_local', 'TOV_local', 'PF_local', 'PLUS_MINUS_local',
       'TEAM_ABBREVIATION_visitante', 'equipo_visitante', 'WL_visitante',
       'pts_visitante', 'FGM_visitante', 'FGA_visitante', 'FG_PCT_visitante',
       'FG3M_visitante', 'FG3A_visitante', 'FG3_PCT_visitante',
       'FTM_visitante', 'FTA_visitante', 'FT_PCT_visitante', 'OREB_visitante',
       'DREB_visitante', 'REB_visitante', 'AST_visitante', 'STL_visitante',
       'BLK_visitante', 'TOV_visitante', 'PF_visitante',
       'PLUS_MINUS_visitante'

Index(['abreviación_eq_local', 'equipo_local', 'fecha', 'cabecera', 'WL_local',
       'pts_local', 'FGM_local', 'FGA_local', 'FG_PCT_local', 'FG3M_local',
       'FG3A_local', 'FG3_PCT_local', 'FTM_local', 'FTA_local', 'FT_PCT_local',
       'OREB_local', 'DREB_local', 'REB_local', 'AST_local', 'STL_local',
       'BLK_local', 'TOV_local', 'PF_local', 'PLUS_MINUS_local',
       'TEAM_ABBREVIATION_visitante', 'equipo_visitante', 'WL_visitante',
       'pts_visitante', 'FGM_visitante', 'FGA_visitante', 'FG_PCT_visitante',
       'FG3M_visitante', 'FG3A_visitante', 'FG3_PCT_visitante',
       'FTM_visitante', 'FTA_visitante', 'FT_PCT_visitante', 'OREB_visitante',
       'DREB_visitante', 'REB_visitante', 'AST_visitante', 'STL_visitante',
       'BLK_visitante', 'TOV_visitante', 'PF_visitante',
       'PLUS_MINUS_visitante'],
      dtype='object')

### AÑADIR CON LOS DATOS DE  EQUIPO LOCAL

In [293]:
# Esta función devuelve las medias de las estadísticas que ha hecho un equipo en los últimos 3 partidos.
# únicamente devuelve los valores numéricos.
def info_ultimos_partidos_local(row):
    fecha = row['fecha']
    equipo_local =row['equipo_local']
    resultados=[]
    # Se aplica un filtro a partidos_df con 2 condiciones:
    # 1) que la fecha sea menor a la fecha del partido que entra como parámetro de la función.
    # 2) que el equipo que entra como parámetro esté en la columna equipo_local o en equipo_visitante
    temp_df = partidos_df[(partidos_df['fecha']<fecha) & ((partidos_df['equipo_local']==equipo_local) | (partidos_df['equipo_visitante']==equipo_local))]
    
    # Se ordenan las filas por la columna fecha y se queda sólo con las 3 últimas.
    temp_df=temp_df.sort_values(by=['fecha'])[-3:]
    
    #se crean listas vacias para los datos que se vana  devolver
    WL_list=[]
    pts_list=[] 
    FGM_list=[] 
    FGA_list=[] 
    FG_PCT_list=[] 
    FG3M_list=[]
    FG3A_list=[] 
    FG3_PCT_list=[] 
    FTM_list=[] 
    FTA_list=[] 
    FT_PCT_list=[]
    OREB_list=[]
    DREB_list=[] 
    REB_list=[] 
    AST_list=[] 
    STL_list=[]
    BLK_list=[] 
    TOV_list=[] 
    PF_list=[] 
    PLUS_MINUS_list=[]
    
    # Se recorren las 3 filas filas seleccionadas 
    # y se comprueba si el equipo sobre el que vamos a rellenar los datos se encuentra en como local o visitante
    if len(temp_df) >=3 :
        for fila in range(3):
            if temp_df['equipo_local'].iloc[fila]==equipo_local:
                (abreviación_eq_local, equipo_local, fecha, cabecera, WL_local,
        pts_local, FGM_local, FGA_local, FG_PCT_local, FG3M_local,
        FG3A_local, FG3_PCT_local, FTM_local, FTA_local, FT_PCT_local,
        OREB_local, DREB_local, REB_local, AST_local, STL_local,
        BLK_local, TOV_local, PF_local, PLUS_MINUS_local,
        TEAM_ABBREVIATION_visitante, equipo_visitante, WL_visitante,
        pts_visitante, FGM_visitante, FGA_visitante, FG_PCT_visitante,
        FG3M_visitante, FG3A_visitante, FG3_PCT_visitante,
        FTM_visitante, FTA_visitante, FT_PCT_visitante, OREB_visitante,
        DREB_visitante, REB_visitante, AST_visitante, STL_visitante,
        BLK_visitante, TOV_visitante, PF_visitante,
        PLUS_MINUS_visitante) = temp_df.iloc[fila]
            else:
                (abreviación_eq_tirar, equipo_tirar, fecha, cabecera, WL_tirar,
        pts_tirar, FGM_tirar, FGA_tirar, FG_PCT_tirar, FG3M_tirar,
        FG3A_tirar, FG3_PCT_tirar, FTM_tirar, FTA_tirar, FT_PCT_tirar,
        OREB_tirar, DREB_tirar, REB_tirar, AST_tirar, STL_tirar,
        BLK_tirar, TOV_tirar, PF_tirar, PLUS_MINUS_tirar,
        TEAM_ABBREVIATION_local, equipo_local, WL_local,
        pts_local, FGM_local, FGA_local, FG_PCT_local,
        FG3M_local, FG3A_local, FG3_PCT_local,
        FTM_local, FTA_local, FT_PCT_local, OREB_local,
        DREB_local, REB_local, AST_local, STL_local,
        BLK_local, TOV_local, PF_local,
        PLUS_MINUS_local) = temp_df.iloc[fila]
            
            # se añaden los datos válidos a las listas
            WL_list.append(WL_local)
            pts_list.append(pts_local)
            FGM_list.append(FGM_local) 
            FGA_list.append(FGA_local) 
            FG_PCT_list.append(FG_PCT_local)
            FG3M_list.append(FG3M_local)
            FG3A_list.append(FG3A_local)
            FG3_PCT_list.append(FG3_PCT_local) 
            FTM_list.append(FTM_local) 
            FTA_list.append(FTA_local)
            FT_PCT_list.append(FT_PCT_local)
            OREB_list.append(OREB_local)
            DREB_list.append(DREB_local)
            REB_list.append(REB_local)
            AST_list.append(AST_local)
            STL_list.append(STL_local)
            BLK_list.append(BLK_local) 
            TOV_list.append(TOV_local) 
            PF_list.append(PF_local)
            PLUS_MINUS_list.append(PLUS_MINUS_local)
    
    elif len(temp_df) < 3 :
        for fila in range(len(temp_df)):
            if temp_df['equipo_local'].iloc[fila]==equipo_local:
                (abreviación_eq_local, equipo_local, fecha, cabecera, WL_local,
        pts_local, FGM_local, FGA_local, FG_PCT_local, FG3M_local,
        FG3A_local, FG3_PCT_local, FTM_local, FTA_local, FT_PCT_local,
        OREB_local, DREB_local, REB_local, AST_local, STL_local,
        BLK_local, TOV_local, PF_local, PLUS_MINUS_local,
        TEAM_ABBREVIATION_visitante, equipo_visitante, WL_visitante,
        pts_visitante, FGM_visitante, FGA_visitante, FG_PCT_visitante,
        FG3M_visitante, FG3A_visitante, FG3_PCT_visitante,
        FTM_visitante, FTA_visitante, FT_PCT_visitante, OREB_visitante,
        DREB_visitante, REB_visitante, AST_visitante, STL_visitante,
        BLK_visitante, TOV_visitante, PF_visitante,
        PLUS_MINUS_visitante) = temp_df.iloc[fila]
            else:
                (abreviación_eq_tirar, equipo_tirar, fecha, cabecera, WL_tirar,
        pts_tirar, FGM_tirar, FGA_tirar, FG_PCT_tirar, FG3M_tirar,
        FG3A_tirar, FG3_PCT_tirar, FTM_tirar, FTA_tirar, FT_PCT_tirar,
        OREB_tirar, DREB_tirar, REB_tirar, AST_tirar, STL_tirar,
        BLK_tirar, TOV_tirar, PF_tirar, PLUS_MINUS_tirar,
        TEAM_ABBREVIATION_local, equipo_local, WL_local,
        pts_local, FGM_local, FGA_local, FG_PCT_local,
        FG3M_local, FG3A_local, FG3_PCT_local,
        FTM_local, FTA_local, FT_PCT_local, OREB_local,
        DREB_local, REB_local, AST_local, STL_local,
        BLK_local, TOV_local, PF_local,
        PLUS_MINUS_local) = temp_df.iloc[fila]
            
            # se añaden los datos válidos a las listas
            WL_list.append(WL_local)
            pts_list.append(pts_local)
            FGM_list.append(FGM_local) 
            FGA_list.append(FGA_local) 
            FG_PCT_list.append(FG_PCT_local)
            FG3M_list.append(FG3M_local)
            FG3A_list.append(FG3A_local)
            FG3_PCT_list.append(FG3_PCT_local) 
            FTM_list.append(FTM_local) 
            FTA_list.append(FTA_local)
            FT_PCT_list.append(FT_PCT_local)
            OREB_list.append(OREB_local)
            DREB_list.append(DREB_local)
            REB_list.append(REB_local)
            AST_list.append(AST_local)
            STL_list.append(STL_local)
            BLK_list.append(BLK_local) 
            TOV_list.append(TOV_local) 
            PF_list.append(PF_local)
            PLUS_MINUS_list.append(PLUS_MINUS_local)
    
    # se calculan las medias de los valores guardados en las listas  
    
    pts=np.mean(pts_list)
    FGM=np.mean(FGM_list)
    FGA=np.mean(FGA_list)
    FG_PCT=np.mean(FG_PCT_list)
    FG3M=np.mean(FG3M_list)
    FG3A=np.mean(FG3A_list)
    FG3_PCT=np.mean(FG3_PCT_list)
    FTM=np.mean(FTM_list)
    FTA=np.mean(FTA_list) 
    FT_PCT=np.mean(FT_PCT_list)
    OREB=np.mean(OREB_list)
    DREB=np.mean(DREB_list)
    REB=np.mean(REB_list)
    AST=np.mean(AST_list) 
    STL=np.mean(STL_list)
    BLK=np.mean(BLK_list)
    TOV=np.mean(TOV_list)
    PF=np.mean(PF_list)
    PLUS_MINUS=np.mean(PLUS_MINUS_list)
    lista_extend=[pts , FGM, FGA, FG_PCT, FG3M,FG3A, FG3_PCT, FTM, FTA, FT_PCT,OREB, DREB, REB, AST, STL,BLK, TOV, PF, PLUS_MINUS]
    resultados.extend(lista_extend)
   
    return resultados

resultado_funcion= predictor_df.apply(info_ultimos_partidos_local,axis=1,raw=False)
columnas=['pts_local', 'FGM_local', 'FGA_local', 'FG_PCT_local', 'FG3M_local','FG3A_local', 'FG3_PCT_local', 'FTM_local', 'FTA_local', 'FT_PCT_local','OREB_local', 'DREB_local', 'REB_local', 'AST_local', 'STL_local','BLK_local', 'TOV_local', 'PF_local', 'PLUS_MINUS_local']
pts = [] 
FGM = [] 
FGA = [] 
FG_PCT = [] 
FG3M = []
FG3A = []
FG3_PCT = []
FTM = []
FTA = []
FT_PCT = []
OREB = []
DREB = []
REB = []
AST = []
STL = []
BLK = []
TOV = []
PF = []
PLUS_MINUS = []

for i in range(len(resultado_funcion)):
    pts.append(resultado_funcion[i][0]) 
    FGM.append(resultado_funcion[i][1])
    FGA.append(resultado_funcion[i][2]) 
    FG_PCT.append(resultado_funcion[i][3])
    FG3M.append(resultado_funcion[i][4])
    FG3A.append(resultado_funcion[i][5])
    FG3_PCT.append(resultado_funcion[i][6]) 
    FTM.append(resultado_funcion[i][7])
    FTA.append(resultado_funcion[i][8])
    FT_PCT.append(resultado_funcion[i][9])
    OREB.append(resultado_funcion[i][10])
    DREB.append(resultado_funcion[i][11])
    REB.append(resultado_funcion[i][12])
    AST.append(resultado_funcion[i][13])
    STL.append(resultado_funcion[i][14])
    BLK.append(resultado_funcion[i][15])
    TOV.append(resultado_funcion[i][16])
    PF.append(resultado_funcion[i][17])
    PLUS_MINUS.append(resultado_funcion[i][18])
prueba_df= pd.DataFrame({'pts_local':pts, 'FGM_local':FGM, 'FGA_local':FGA, 'FG_PCT_local':FG_PCT, 'FG3M_local':FG3M,'FG3A_local':FG3A, 'FG3_PCT_local':FG3_PCT, 'FTM_local':FTM, 'FTA_local':FTA, 'FT_PCT_local':FT_PCT,'OREB_local':OREB, 'DREB_local':DREB, 'REB_local':REB, 'AST_local':AST, 'STL_local':STL,'BLK_local':BLK, 'TOV_local':TOV, 'PF_local':PF, 'PLUS_MINUS_local':PLUS_MINUS})
predictor_df=pd.concat([predictor_df, prueba_df], axis=1)

    

C:\Users\iauzm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\iauzm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


### AÑADIR COLUMNAS CON DATOS DEL EQUIPO VISITANTE

In [294]:
# Esta función devuelve las medias de las estadísticas que ha hecho un equipo en los últimos 3 partidos.
# únicamente devuelve los valores numéricos.
def info_ultimos_partidos_visitante(row):
    fecha = row['fecha']
    equipo_visitante =row['equipo_visitante']
    resultados=[]
    # Se aplica un filtro a partidos_df con 2 condiciones:
    # 1) que la fecha sea menor a la fecha del partido que entra como parámetro de la función.
    # 2) que el equipo que entra como parámetro esté en la columna equipo_visitante o en equipo_visitante
    temp_df = partidos_df[(partidos_df['fecha']<fecha) & ((partidos_df['equipo_visitante']==equipo_visitante) | (partidos_df['equipo_visitante']==equipo_visitante))]
    
    # Se ordenan las filas por la columna fecha y se queda sólo con las 3 últimas.
    temp_df=temp_df.sort_values(by=['fecha'])[-3:]
    
    #se crean listas vacias para los datos que se vana  devolver
    WL_list=[]
    pts_list=[] 
    FGM_list=[] 
    FGA_list=[] 
    FG_PCT_list=[] 
    FG3M_list=[]
    FG3A_list=[] 
    FG3_PCT_list=[] 
    FTM_list=[] 
    FTA_list=[] 
    FT_PCT_list=[]
    OREB_list=[]
    DREB_list=[] 
    REB_list=[] 
    AST_list=[] 
    STL_list=[]
    BLK_list=[] 
    TOV_list=[] 
    PF_list=[] 
    PLUS_MINUS_list=[]
    
    # Se recorren las 3 filas filas seleccionadas 
    # y se comprueba si el equipo sobre el que vamos a rellenar los datos se encuentra en como visitante o visitante
    if len(temp_df) >=3 :
        for fila in range(3):
            if temp_df['equipo_visitante'].iloc[fila]==equipo_visitante:
                (abreviación_eq_visitante, equipo_visitante, fecha, cabecera, WL_visitante,
        pts_visitante, FGM_visitante, FGA_visitante, FG_PCT_visitante, FG3M_visitante,
        FG3A_visitante, FG3_PCT_visitante, FTM_visitante, FTA_visitante, FT_PCT_visitante,
        OREB_visitante, DREB_visitante, REB_visitante, AST_visitante, STL_visitante,
        BLK_visitante, TOV_visitante, PF_visitante, PLUS_MINUS_visitante,
        TEAM_ABBREVIATION_visitante, equipo_visitante, WL_visitante,
        pts_visitante, FGM_visitante, FGA_visitante, FG_PCT_visitante,
        FG3M_visitante, FG3A_visitante, FG3_PCT_visitante,
        FTM_visitante, FTA_visitante, FT_PCT_visitante, OREB_visitante,
        DREB_visitante, REB_visitante, AST_visitante, STL_visitante,
        BLK_visitante, TOV_visitante, PF_visitante,
        PLUS_MINUS_visitante) = temp_df.iloc[fila]
            else:
                (abreviación_eq_tirar, equipo_tirar, fecha, cabecera, WL_tirar,
        pts_tirar, FGM_tirar, FGA_tirar, FG_PCT_tirar, FG3M_tirar,
        FG3A_tirar, FG3_PCT_tirar, FTM_tirar, FTA_tirar, FT_PCT_tirar,
        OREB_tirar, DREB_tirar, REB_tirar, AST_tirar, STL_tirar,
        BLK_tirar, TOV_tirar, PF_tirar, PLUS_MINUS_tirar,
        TEAM_ABBREVIATION_visitante, equipo_visitante, WL_visitante,
        pts_visitante, FGM_visitante, FGA_visitante, FG_PCT_visitante,
        FG3M_visitante, FG3A_visitante, FG3_PCT_visitante,
        FTM_visitante, FTA_visitante, FT_PCT_visitante, OREB_visitante,
        DREB_visitante, REB_visitante, AST_visitante, STL_visitante,
        BLK_visitante, TOV_visitante, PF_visitante,
        PLUS_MINUS_visitante) = temp_df.iloc[fila]
            
            # se añaden los datos válidos a las listas
            WL_list.append(WL_visitante)
            pts_list.append(pts_visitante)
            FGM_list.append(FGM_visitante) 
            FGA_list.append(FGA_visitante) 
            FG_PCT_list.append(FG_PCT_visitante)
            FG3M_list.append(FG3M_visitante)
            FG3A_list.append(FG3A_visitante)
            FG3_PCT_list.append(FG3_PCT_visitante) 
            FTM_list.append(FTM_visitante) 
            FTA_list.append(FTA_visitante)
            FT_PCT_list.append(FT_PCT_visitante)
            OREB_list.append(OREB_visitante)
            DREB_list.append(DREB_visitante)
            REB_list.append(REB_visitante)
            AST_list.append(AST_visitante)
            STL_list.append(STL_visitante)
            BLK_list.append(BLK_visitante) 
            TOV_list.append(TOV_visitante) 
            PF_list.append(PF_visitante)
            PLUS_MINUS_list.append(PLUS_MINUS_visitante)
    
    elif len(temp_df) < 3 :
        for fila in range(len(temp_df)):
            if temp_df['equipo_visitante'].iloc[fila]==equipo_visitante:
                (abreviación_eq_visitante, equipo_visitante, fecha, cabecera, WL_visitante,
        pts_visitante, FGM_visitante, FGA_visitante, FG_PCT_visitante, FG3M_visitante,
        FG3A_visitante, FG3_PCT_visitante, FTM_visitante, FTA_visitante, FT_PCT_visitante,
        OREB_visitante, DREB_visitante, REB_visitante, AST_visitante, STL_visitante,
        BLK_visitante, TOV_visitante, PF_visitante, PLUS_MINUS_visitante,
        TEAM_ABBREVIATION_visitante, equipo_visitante, WL_visitante,
        pts_visitante, FGM_visitante, FGA_visitante, FG_PCT_visitante,
        FG3M_visitante, FG3A_visitante, FG3_PCT_visitante,
        FTM_visitante, FTA_visitante, FT_PCT_visitante, OREB_visitante,
        DREB_visitante, REB_visitante, AST_visitante, STL_visitante,
        BLK_visitante, TOV_visitante, PF_visitante,
        PLUS_MINUS_visitante) = temp_df.iloc[fila]
            else:
                (abreviación_eq_tirar, equipo_tirar, fecha, cabecera, WL_tirar,
        pts_tirar, FGM_tirar, FGA_tirar, FG_PCT_tirar, FG3M_tirar,
        FG3A_tirar, FG3_PCT_tirar, FTM_tirar, FTA_tirar, FT_PCT_tirar,
        OREB_tirar, DREB_tirar, REB_tirar, AST_tirar, STL_tirar,
        BLK_tirar, TOV_tirar, PF_tirar, PLUS_MINUS_tirar,
        TEAM_ABBREVIATION_visitante, equipo_visitante, WL_visitante,
        pts_visitante, FGM_visitante, FGA_visitante, FG_PCT_visitante,
        FG3M_visitante, FG3A_visitante, FG3_PCT_visitante,
        FTM_visitante, FTA_visitante, FT_PCT_visitante, OREB_visitante,
        DREB_visitante, REB_visitante, AST_visitante, STL_visitante,
        BLK_visitante, TOV_visitante, PF_visitante,
        PLUS_MINUS_visitante) = temp_df.iloc[fila]
            
            # se añaden los datos válidos a las listas
            WL_list.append(WL_visitante)
            pts_list.append(pts_visitante)
            FGM_list.append(FGM_visitante) 
            FGA_list.append(FGA_visitante) 
            FG_PCT_list.append(FG_PCT_visitante)
            FG3M_list.append(FG3M_visitante)
            FG3A_list.append(FG3A_visitante)
            FG3_PCT_list.append(FG3_PCT_visitante) 
            FTM_list.append(FTM_visitante) 
            FTA_list.append(FTA_visitante)
            FT_PCT_list.append(FT_PCT_visitante)
            OREB_list.append(OREB_visitante)
            DREB_list.append(DREB_visitante)
            REB_list.append(REB_visitante)
            AST_list.append(AST_visitante)
            STL_list.append(STL_visitante)
            BLK_list.append(BLK_visitante) 
            TOV_list.append(TOV_visitante) 
            PF_list.append(PF_visitante)
            PLUS_MINUS_list.append(PLUS_MINUS_visitante)
    
    # se calculan las medias de los valores guardados en las listas  
    
    pts=np.mean(pts_list)
    FGM=np.mean(FGM_list)
    FGA=np.mean(FGA_list)
    FG_PCT=np.mean(FG_PCT_list)
    FG3M=np.mean(FG3M_list)
    FG3A=np.mean(FG3A_list)
    FG3_PCT=np.mean(FG3_PCT_list)
    FTM=np.mean(FTM_list)
    FTA=np.mean(FTA_list) 
    FT_PCT=np.mean(FT_PCT_list)
    OREB=np.mean(OREB_list)
    DREB=np.mean(DREB_list)
    REB=np.mean(REB_list)
    AST=np.mean(AST_list) 
    STL=np.mean(STL_list)
    BLK=np.mean(BLK_list)
    TOV=np.mean(TOV_list)
    PF=np.mean(PF_list)
    PLUS_MINUS=np.mean(PLUS_MINUS_list)
    lista_extend=[pts , FGM, FGA, FG_PCT, FG3M,FG3A, FG3_PCT, FTM, FTA, FT_PCT,OREB, DREB, REB, AST, STL,BLK, TOV, PF, PLUS_MINUS]
    resultados.extend(lista_extend)
   
    return resultados
resultado_funcion= predictor_df.apply(info_ultimos_partidos_visitante,axis=1,raw=False)
columnas=['pts_visitante', 'FGM_visitante', 'FGA_visitante', 'FG_PCT_visitante', 'FG3M_visitante','FG3A_visitante', 'FG3_PCT_visitante', 'FTM_visitante', 'FTA_visitante', 'FT_PCT_visitante','OREB_visitante', 'DREB_visitante', 'REB_visitante', 'AST_visitante', 'STL_visitante','BLK_visitante', 'TOV_visitante', 'PF_visitante', 'PLUS_MINUS_visitante']
pts = [] 
FGM = [] 
FGA = [] 
FG_PCT = [] 
FG3M = []
FG3A = []
FG3_PCT = []
FTM = []
FTA = []
FT_PCT = []
OREB = []
DREB = []
REB = []
AST = []
STL = []
BLK = []
TOV = []
PF = []
PLUS_MINUS = []

for i in range(len(resultado_funcion)):
    pts.append(resultado_funcion[i][0]) 
    FGM.append(resultado_funcion[i][1])
    FGA.append(resultado_funcion[i][2]) 
    FG_PCT.append(resultado_funcion[i][3])
    FG3M.append(resultado_funcion[i][4])
    FG3A.append(resultado_funcion[i][5])
    FG3_PCT.append(resultado_funcion[i][6]) 
    FTM.append(resultado_funcion[i][7])
    FTA.append(resultado_funcion[i][8])
    FT_PCT.append(resultado_funcion[i][9])
    OREB.append(resultado_funcion[i][10])
    DREB.append(resultado_funcion[i][11])
    REB.append(resultado_funcion[i][12])
    AST.append(resultado_funcion[i][13])
    STL.append(resultado_funcion[i][14])
    BLK.append(resultado_funcion[i][15])
    TOV.append(resultado_funcion[i][16])
    PF.append(resultado_funcion[i][17])
    PLUS_MINUS.append(resultado_funcion[i][18])
prueba_df= pd.DataFrame({'pts_visitante':pts, 'FGM_visitante':FGM, 'FGA_visitante':FGA, 'FG_PCT_visitante':FG_PCT, 'FG3M_visitante':FG3M,'FG3A_visitante':FG3A, 'FG3_PCT_visitante':FG3_PCT, 'FTM_visitante':FTM, 'FTA_visitante':FTA, 'FT_PCT_visitante':FT_PCT,'OREB_visitante':OREB, 'DREB_visitante':DREB, 'REB_visitante':REB, 'AST_visitante':AST, 'STL_visitante':STL,'BLK_visitante':BLK, 'TOV_visitante':TOV, 'PF_visitante':PF, 'PLUS_MINUS_visitante':PLUS_MINUS})
predictor_df=pd.concat([predictor_df, prueba_df], axis=1)
predictor_df.sample(10)

C:\Users\iauzm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\iauzm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,fecha,equipo_local,equipo_visitante,pts_local,FGM_local,FGA_local,FG_PCT_local,FG3M_local,FG3A_local,FG3_PCT_local,...,FT_PCT_visitante,OREB_visitante,DREB_visitante,REB_visitante,AST_visitante,STL_visitante,BLK_visitante,TOV_visitante,PF_visitante,PLUS_MINUS_visitante
14,2023-10-26,Milwaukee Bucks,Philadelphia 76ers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1213,2024-04-27,Los Angeles Lakers,Denver Nuggets,102.333333,40.333333,82.333333,0.490000,8.666667,28.666667,0.298000,...,0.870000,12.000000,34.000000,46.000000,30.000000,7.333333,4.000000,12.333333,16.666667,7.000000
892,2024-03-06,Golden State Warriors,Milwaukee Bucks,106.000000,41.333333,96.000000,0.429667,13.000000,43.000000,0.299333,...,0.811000,8.666667,38.000000,46.666667,25.000000,7.333333,5.666667,10.666667,16.333333,16.333333
949,2024-03-14,Boston Celtics,Phoenix Suns,120.333333,46.000000,95.333333,0.482000,16.666667,42.666667,0.389333,...,0.757667,14.000000,36.666667,50.666667,26.000000,9.000000,7.000000,11.000000,19.000000,4.000000
228,2023-11-26,Brooklyn Nets,Chicago Bulls,118.666667,44.000000,90.666667,0.485333,13.333333,33.000000,0.403667,...,0.845667,12.666667,28.000000,40.666667,22.000000,7.000000,3.666667,9.333333,23.666667,-12.000000
700,2024-02-03,Atlanta Hawks,Golden State Warriors,131.000000,48.666667,97.000000,0.501333,13.000000,36.333333,0.343000,...,0.787000,8.333333,35.666667,44.000000,27.666667,5.333333,6.000000,13.333333,19.333333,0.000000
564,2024-01-15,Cleveland Cavaliers,Chicago Bulls,114.000000,38.666667,82.333333,0.470333,15.000000,40.000000,0.384000,...,0.798667,10.000000,35.666667,45.666667,27.333333,7.000000,7.333333,11.000000,17.666667,-1.000000
148,2023-11-14,Oklahoma City Thunder,San Antonio Spurs,112.333333,41.000000,85.000000,0.482667,9.666667,28.333333,0.344000,...,0.768333,10.666667,30.333333,41.000000,31.333333,5.333333,5.666667,10.666667,19.333333,-17.000000
834,2024-02-27,Cleveland Cavaliers,Dallas Mavericks,106.666667,38.333333,77.666667,0.496000,9.333333,27.000000,0.354667,...,0.704000,6.666667,34.333333,41.000000,23.333333,7.666667,4.333333,11.000000,12.666667,1.333333
91,2023-11-06,Detroit Pistons,Golden State Warriors,107.666667,41.333333,90.333333,0.458000,10.666667,30.000000,0.356333,...,0.716667,17.000000,31.666667,48.666667,30.000000,8.333333,4.333333,14.000000,21.333333,6.333333


In [285]:
prueba_df= pd.DataFrame({'pts_local':pts, 'FGM_local':FGM, 'FGA_local':FGA, 'FG_PCT_local':FG_PCT, 'FG3M_local':FG3M,'FG3A_local':FG3A, 'FG3_PCT_local':FG3_PCT, 'FTM_local':FTM, 'FTA_local':FTA, 'FT_PCT_local':FT_PCT,'OREB_local':OREB, 'DREB_local':DREB, 'REB_local':REB, 'AST_local':AST, 'STL_local':STL,'BLK_local':BLK, 'TOV_local':TOV, 'PF_local':PF, 'PLUS_MINUS_local':PLUS_MINUS})
pd.concat([predictor_df, prueba_df], axis=1)

,fecha,equipo_local,equipo_visitante,pts_local,FGM_local,FGA_local,FG_PCT_local,FG3M_local,FG3A_local,FG3_PCT_local,...,FT_PCT_local,OREB_local,DREB_local,REB_local,AST_local,STL_local,BLK_local,TOV_local,PF_local,PLUS_MINUS_local
0,2023-10-24,Denver Nuggets,Los Angeles Lakers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-10-24,Golden State Warriors,Phoenix Suns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-10-25,Brooklyn Nets,Cleveland Cavaliers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-10-25,Charlotte Hornets,Atlanta Hawks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-10-25,Chicago Bulls,Oklahoma City Thunder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261,2024-05-24,Minnesota Timberwolves,Dallas Mavericks,104.333333,40.666667,92.000000,0.447000,12.000000,32.666667,0.365333,...,0.722667,11.000000,38.000000,49.000000,24.666667,7.333333,5.666667,12.666667,16.000000,-0.333333
1262,2024-05-25,Indiana Pacers,Boston Celtics,124.333333,46.666667,97.666667,0.474000,15.666667,42.333333,0.373667,...,0.860333,10.666667,38.000000,48.666667,33.333333,6.666667,5.666667,12.000000,22.333333,9.333333
1263,2024-05-26,Dallas Mavericks,Minnesota Timberwolves,125.333333,44.333333,91.333333,0.486333,17.666667,43.333333,0.408333,...,0.758333,9.000000,36.333333,45.333333,22.666667,5.000000,4.000000,12.000000,17.333333,9.000000
1264,2024-05-27,Indiana Pacers,Boston Celtics,124.333333,46.666667,97.666667,0.474000,15.666667,42.333333,0.373667,...,0.860333,10.666667,38.000000,48.666667,33.333333,6.666667,5.666667,12.000000,22.333333,9.333333


In [287]:
#COMPROBACION
fecha=predictor_df['fecha'].iloc[-1]
equipo_local=predictor_df['equipo_local'].iloc[-1]
a = partidos_df[(partidos_df['fecha']<fecha) & ((partidos_df['equipo_local']==equipo_local) | (partidos_df['equipo_visitante']==equipo_local))]
    
# Se ordenan las filas por la columna fecha y se queda sólo con las 3 últimas.
a=a.sort_values(by=['fecha'])[-3:]
a

,abreviación_eq_local,equipo_local,fecha,cabecera,WL_local,pts_local,FGM_local,FGA_local,FG_PCT_local,FG3M_local,...,FT_PCT_visitante,OREB_visitante,DREB_visitante,REB_visitante,AST_visitante,STL_visitante,BLK_visitante,TOV_visitante,PF_visitante,PLUS_MINUS_visitante
1259,MIN,Minnesota Timberwolves,2024-05-22,MIN VS DAL,L,105.0,38.0,89.0,0.427,18.0,...,0.941,11.0,37.0,48.0,21.0,5.0,8.0,13.0,15.0,3.0
1261,MIN,Minnesota Timberwolves,2024-05-24,MIN VS DAL,L,108.0,35.0,85.0,0.412,12.0,...,0.682,11.0,33.0,44.0,24.0,7.0,7.0,10.0,17.0,1.0
1263,DAL,Dallas Mavericks,2024-05-26,DAL VS MIN,W,116.0,38.0,68.0,0.559,14.0,...,0.706,10.0,23.0,33.0,26.0,4.0,2.0,10.0,24.0,-9.0


### CREAR TARGET
El target será la columna cganaor_local, contendrá 1 y si ganó el euipo local y 0 si ganó el visitante.
Como son datos reales del partido, tengo que sacarlo del dataset de partidos 2024. 

In [312]:
partidos_df['ganador_local']=partidos_df['pts_local'] - partidos_df['pts_visitante'] 
partidos_df['ganador_local'] = partidos_df['ganador_local'].apply(lambda x: 1 if x>0 else 0)

In [315]:
partidos_df[['pts_local','pts_visitante','ganador_local']].head()

,pts_local,pts_visitante,ganador_local
0,119.0,107.0,1
1,104.0,108.0,0
2,113.0,114.0,0
3,116.0,110.0,1
4,104.0,124.0,0


In [317]:
print(predictor_df.shape)
print(partidos_df.shape)

(1266, 41)
(1266, 47)


#### Merge.
Lo haré csbre la columna fecha y equipo local, y sólo añadiré la columna ganador_local

In [318]:
df_para_merge = partidos_df[['fecha','equipo_local','ganador_local']]
df_pred_merged = predictor_df.merge(df_para_merge,on=['fecha','equipo_local'],how='outer')
df_pred_merged

,fecha,equipo_local,equipo_visitante,pts_local,FGM_local,FGA_local,FG_PCT_local,FG3M_local,FG3A_local,FG3_PCT_local,...,OREB_visitante,DREB_visitante,REB_visitante,AST_visitante,STL_visitante,BLK_visitante,TOV_visitante,PF_visitante,PLUS_MINUS_visitante,ganador_local
0,2023-10-24,Denver Nuggets,Los Angeles Lakers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2023-10-24,Golden State Warriors,Phoenix Suns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2023-10-25,Brooklyn Nets,Cleveland Cavaliers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2023-10-25,Charlotte Hornets,Atlanta Hawks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,2023-10-25,Chicago Bulls,Oklahoma City Thunder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261,2024-05-24,Minnesota Timberwolves,Dallas Mavericks,106.000000,36.333333,84.666667,0.428000,14.333333,41.000000,0.345333,...,10.666667,35.333333,46.000000,26.000000,6.000000,6.333333,12.333333,17.666667,8.000000,0
1262,2024-05-25,Indiana Pacers,Boston Celtics,122.666667,50.000000,87.333333,0.576667,12.333333,29.333333,0.430667,...,10.000000,36.000000,46.000000,19.333333,4.666667,3.666667,11.666667,15.333333,11.333333,0
1263,2024-05-26,Dallas Mavericks,Minnesota Timberwolves,111.333333,42.000000,84.333333,0.498000,11.333333,30.000000,0.366000,...,9.666667,31.000000,40.666667,25.333333,9.000000,8.333333,11.000000,20.333333,6.333333,1
1264,2024-05-27,Indiana Pacers,Boston Celtics,116.333333,47.666667,91.333333,0.521333,9.666667,28.666667,0.325667,...,8.666667,34.000000,42.666667,20.333333,5.333333,6.666667,10.666667,14.666667,7.666667,0


### CONTROL DE NANS
En las primeras 50 columnas se generan nans, debido son partidos en los que un equipo juega por primera vez, y no puede crear medias de los resultados obtenidos en partidos anteriores. 

OPCIONES: 
* Me cargo esas columnas
* Las lleno con la media
* Las relleno con valores obtenidos por KNN
* Cambio el script para incluir los últimos partidos de la temporada anteior y usarlo  para calcular estos primeros partidos, yya que va por fecha. 

Voy a probar primero eliminándolos sin más aunque creo que al fina tendré que aprovechar toda la información posible, y necesitaré rellenarlos con el KNN.

In [319]:
df_1 = df_pred_merged.dropna()
df_1.shape

(1230, 42)

In [320]:
df_1.head()

,fecha,equipo_local,equipo_visitante,pts_local,FGM_local,FGA_local,FG_PCT_local,FG3M_local,FG3A_local,FG3_PCT_local,...,OREB_visitante,DREB_visitante,REB_visitante,AST_visitante,STL_visitante,BLK_visitante,TOV_visitante,PF_visitante,PLUS_MINUS_visitante,ganador_local
13,2023-10-26,Los Angeles Lakers,Phoenix Suns,107.0,41.0,90.0,0.456,10.0,29.0,0.345,...,17.0,43.0,60.0,23.0,5.0,7.0,19.0,22.0,4.0,1
17,2023-10-27,Charlotte Hornets,Detroit Pistons,116.0,43.0,86.0,0.500,11.0,37.0,0.297,...,17.0,39.0,56.0,28.0,3.0,13.0,16.0,23.0,-1.0,0
19,2023-10-27,Cleveland Cavaliers,Oklahoma City Thunder,114.0,44.0,94.0,0.468,17.0,43.0,0.395,...,5.0,38.0,43.0,30.0,6.0,4.0,15.0,19.0,20.0,0
24,2023-10-27,San Antonio Spurs,Houston Rockets,119.0,46.0,89.0,0.517,11.0,35.0,0.314,...,7.0,24.0,31.0,19.0,6.0,3.0,18.0,24.0,-30.0,1
28,2023-10-28,Minnesota Timberwolves,Miami Heat,94.0,34.0,100.0,0.340,8.0,31.0,0.258,...,11.0,34.0,45.0,24.0,8.0,6.0,12.0,16.0,-8.0,1


In [323]:
df_1.to_csv('datos_temp')

### UNIR LAS TABLAS CON DATOS GENERALES Y LOS TEMPORALES


In [326]:
gen_df = pd.read_csv('datos_generales',index_col='Unnamed: 0')
temp_df = pd.read_csv('datos_temp',index_col='Unnamed: 0')

In [336]:
gen_df.drop(columns=['wins','loses','abbreviation'],inplace=True)

gen_df.head()

,INS,OUT,Team_full,bpi,off,def,pbpi,OVR,DEF,TIER_T1,TIER_T2,TIER_T3
0,61,82,Atlanta Hawks,-1.485,0.922,-2.407,-1.485,82,64,False,False,True
1,81,91,Boston Celtics,8.075,6.593,1.482,8.075,86,69,True,False,False
2,68,77,Brooklyn Nets,-4.156,-3.599,-0.558,-4.156,80,66,False,False,True
3,66,81,Charlotte Hornets,-7.752,-4.775,-2.977,-7.752,80,61,False,False,True
4,76,85,Chicago Bulls,-0.836,0.598,-1.434,-0.836,82,66,False,False,True


In [328]:
temp_df.head()

,fecha,equipo_local,equipo_visitante,pts_local,FGM_local,FGA_local,FG_PCT_local,FG3M_local,FG3A_local,FG3_PCT_local,...,OREB_visitante,DREB_visitante,REB_visitante,AST_visitante,STL_visitante,BLK_visitante,TOV_visitante,PF_visitante,PLUS_MINUS_visitante,ganador_local
13,2023-10-26,Los Angeles Lakers,Phoenix Suns,107.0,41.0,90.0,0.456,10.0,29.0,0.345,...,17.0,43.0,60.0,23.0,5.0,7.0,19.0,22.0,4.0,1
17,2023-10-27,Charlotte Hornets,Detroit Pistons,116.0,43.0,86.0,0.500,11.0,37.0,0.297,...,17.0,39.0,56.0,28.0,3.0,13.0,16.0,23.0,-1.0,0
19,2023-10-27,Cleveland Cavaliers,Oklahoma City Thunder,114.0,44.0,94.0,0.468,17.0,43.0,0.395,...,5.0,38.0,43.0,30.0,6.0,4.0,15.0,19.0,20.0,0
24,2023-10-27,San Antonio Spurs,Houston Rockets,119.0,46.0,89.0,0.517,11.0,35.0,0.314,...,7.0,24.0,31.0,19.0,6.0,3.0,18.0,24.0,-30.0,1
28,2023-10-28,Minnesota Timberwolves,Miami Heat,94.0,34.0,100.0,0.340,8.0,31.0,0.258,...,11.0,34.0,45.0,24.0,8.0,6.0,12.0,16.0,-8.0,1


In [344]:
pred_df =temp_df.merge(gen_df,left_on='equipo_local',right_on='Team_full' ,suffixes=('_local','XXXX'))

In [345]:
pred_df = pred_df.rename(columns={'INS':'INS_local', 'OUT':'OUT_local',
       'bpi':'bpi_local', 'off':'off_local', 'def':'def_local', 'pbpi':'pbpi_local'
       , 'OVR':'OVR_local', 'DEF':'DEF_local', 'TIER_T1':'TIER_1_local', 'TIER_T2':'TIER_2_local', 'TIER_T3':'TIER__local'})

In [346]:
pred_df=pred_df.merge(gen_df,left_on='equipo_visitante',right_on='Team_full' ,suffixes=('_visitante','XXXX'))
pred_df = pred_df.rename(columns={'INS':'INS_visitante', 'OUT':'OUT_visitante',
       'bpi':'bpi_visitante', 'off':'off_visitante', 'def':'def_visitante', 'pbpi':'pbpi_visitante'
       , 'OVR':'OVR_visitante', 'DEF':'DEF_visitante', 'TIER_T1':'TIER_1_visitante', 'TIER_T2':'TIER_2_visitante', 'TIER_T3':'TIER__visitante'})

In [351]:
pred_df.drop(columns=['fecha','equipo_local','equipo_visitante','Team_fullXXXX','Team_full_visitante'],inplace=True)

pred_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187 entries, 0 to 1186
Data columns (total 61 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   pts_local             1187 non-null   float64
 1   FGM_local             1187 non-null   float64
 2   FGA_local             1187 non-null   float64
 3   FG_PCT_local          1187 non-null   float64
 4   FG3M_local            1187 non-null   float64
 5   FG3A_local            1187 non-null   float64
 6   FG3_PCT_local         1187 non-null   float64
 7   FTM_local             1187 non-null   float64
 8   FTA_local             1187 non-null   float64
 9   FT_PCT_local          1187 non-null   float64
 10  OREB_local            1187 non-null   float64
 11  DREB_local            1187 non-null   float64
 12  REB_local             1187 non-null   float64
 13  AST_local             1187 non-null   float64
 14  STL_local             1187 non-null   float64
 15  BLK_local            

In [352]:
pred_df.to_csv(r'C:\Users\iauzm\OneDrive\Desktop\DATA SCIENCE\AUZMENDI\PROYECTO-ML\modelos\tabla_pred_1')

KeyError: "None of [Index(['equipo_local', 'TIER_1_local'], dtype='object')] are in the [columns]"